In [14]:
import tensorflow as tf
import numpy as np
import pandas as pd
import tensorflow.compat.v2.feature_column as fc
import matplotlib.pyplot as plt
import sys
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [15]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Single Layer

In [ ]:
x = tf.placeholder("float", [None, 784])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [ ]:
y = tf.nn.softmax(tf.matmul(x,W)+b)
y_ = tf.placeholder("float", [None,10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y)) #loss function
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy) #choose optimizer and load loss function

In [ ]:
#init all var
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [ ]:
# train data
for i in range(500): #epoch
    batch_xs, batch_ys = mnist.train.next_batch(100) #generate 100 data for one epoch
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys}) #feed 

In [ ]:
# test
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

# Example

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

x = tf.placeholder(dtype="float", shape=[None, 784])
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

y_ = tf.placeholder("float", shape=[None, 10])
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i%100 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                x:batch[0], y_: batch[1], keep_prob: 1.0})
            print (f"step {i}, training accuracy {train_accuracy}")
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    test_acc = accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0})
    print(f"test accuracy {test_acc}")

# Multi layer

In [26]:
def conv2d(x, output_channel, kernel, strides):
    """
    input kernel size [length, width]
    strides int
    """
    #kernal size for 2d [length, width, channel, features]
    kernel_shape = [kernel[0], kernel[1], x.get_shape()[-1].value, output_channel]
    initial_W = tf.truncated_normal(kernel_shape, stddev=0.1) #std some what IMPORTANT
    W = tf.Variable(initial_W)
    
    #Bias size = output_channel
    initial_B = tf.constant(0.1, shape=[output_channel])
    B = tf.Variable(initial_B)
    #build layer + Bias
    layer = tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")

    #use activation func
    conv_act_layer = tf.nn.relu(layer + B)

    return conv_act_layer

In [27]:
def max_pool(x, kernel, strides):
    """
    input kernel size [length, width]
    strides int
    """
    max_pool_layer = tf.nn.max_pool(x, 
                       ksize=[1, kernel[0], kernel[1], 1], 
                       strides= [1, strides, strides, 1], 
                       padding="SAME")

    return max_pool_layer

In [31]:
def dense_layer(x, output_channel):
    # weight
    shape = [x.get_shape()[-1].value, output_channel]
    initial_W = tf.truncated_normal(shape, stddev=0.1) #std some what IMPORTANT
    W = tf.Variable(initial_W)
    #bias
    initial_B = tf.constant(0.1, shape=[output_channel])
    B = tf.Variable(initial_B)

    layer = tf.matmul(x, W) # (W * x)
    act_layer = tf.nn.relu(layer + B)

    return act_layer

In [32]:
def softmax_layer(x, output_channel):
    # weight
    shape = [x.get_shape()[-1].value, output_channel]
    initial_W = tf.truncated_normal(shape, stddev=0.1) #std some what IMPORTANT
    W = tf.Variable(initial_W)
    #bias
    initial_B = tf.constant(0.1, shape=[output_channel])
    B = tf.Variable(initial_B)

    layer = tf.matmul(x, W)
    act_layer = tf.nn.softmax(layer+B)

    return act_layer

In [33]:
#input placeholder
x = tf.placeholder(dtype="float", shape=[None, 784])
input_x = tf.reshape(x, shape=[-1, 28, 28, 1])

#model network
conv2d_1 = conv2d(input_x, 32, [5,5], 1)
max_pool1 = max_pool(conv2d_1, [2,2], 2)
conv2d_2 = conv2d(max_pool1, 64, [5,5], 1)
max_pool2 = max_pool(conv2d_2, [2,2], 2)
flatten = tf.reshape(max_pool2, [-1, max_pool2.get_shape()[1]*max_pool2.get_shape()[2]*max_pool2.get_shape()[3]])
dense_1 = dense_layer(flatten, 1024)
output = softmax_layer(dense_1, 10) #output = 10

**Equation of cross entropy**  
L = label  
p = predicted probability  
n = classes  
$CE = \sum_{i=1}^n -(L* \log(p)) $

hint: $(L*\log(p))$ only the right class's probability will remain ex: $1*0.5 + 0*0.25 + 0*0.25$

In [34]:
#loss function
#softmax function predict the output probability 0 to 1, can use cross entropy to evaluate the performance
y_ = tf.placeholder("float", shape=[None, 10]) #label
cross_entropy = -1 * tf.reduce_sum(y_*tf.log(output)) #reduce_sum sum all the element

#add optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#test evaluation function (accuracy)
#tf.argmax will out put the "INDEX" of maxium value (the heightest probability)
#tf.equal will generate [True, True, False, ....] matrix

acc_matrix = tf.equal(tf.argmax(output, 1), tf.argmax(y_, 1))

#tf.cast will transform input to required type in here Bool to (0,1)
#tf.reduce_mean will output the mean of matrix

acc_eval = tf.reduce_mean(tf.cast(acc_matrix, "float"))

In [35]:
# Train
epoch = 5000
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epoch):
        batch = mnist.train.next_batch(50)
        #visualize ACC every 50 epoch
        if i % 50 == 0:
            train_acc = acc_eval.eval(feed_dict={x: batch[0], y_:batch[1]})
            print(f"Steps{i}: ACC: {train_acc}")
        #train network
        train_step.run(feed_dict={x: batch[0], y_:batch[1]})
    test_acc = acc_eval.eval(feed_dict={x: mnist.test.images, y_:mnist.test.labels})
    print(f"Network ACC: {test_acc}")

Steps0: ACC: 0.07999999821186066
Steps50: ACC: 0.8600000143051147
Steps100: ACC: 0.8199999928474426
Steps150: ACC: 0.8999999761581421
Steps200: ACC: 0.8799999952316284


KeyboardInterrupt: 

In [36]:
prediction = tf.constant([[1.0, 4.0, 2.0],[1.0, 2.0, 3.0],[2.0, 3.0, 1.0]])
ground_truth = tf.constant([[1.0, 1.0, 1.0],[2.0, 2.0, 2.0],[1.0, 1.0, 1.0]])
Square_Error = tf.square(ground_truth -prediction)
mse = tf.reduce_mean(Square_Error) 

In [39]:
with tf.Session() as sess:
    print(sess.run(mse))

1.8888888
